In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!cp drive/MyDrive/MMDS-data/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
! echo $SPARK_HOME

/content/spark-3.1.1-bin-hadoop3.2


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark import SparkContext

sc = SparkContext("local", "4a")

In [ ]:
sc = SparkContext.getOrCreate("local")

In [ ]:
data = sc.textFile('/content/WHO-COVID-19-global.csv')
header = data.first()
data = data.filter(lambda line:line != header)

In [ ]:
def preprocessLine(x):
  items = x.strip().split(',')[1:]
  region = items[0]
  values = [float(it) for it in items[1:]]
  return (region, values)

import numpy as np
def sumAll(x, y):
  x = np.array(x)
  y = np.array(y)
  return list(x + y)



In [ ]:
!rm -rf result/
result = data.map(preprocessLine).reduceByKey(sumAll)

In [ ]:
for i in result.collect():
  print(i)

('Americas', [176342137.0, 1208742.0069999995, 696441.0, 4434.753, 28836.0, 2821706.0, 9417.268, 4500.0, 13.143, 136.0])
('South-East Asia', [60084208.0, 65053.017, 82449.0, 61.922999999999995, 10916.0, 796201.0, 385.42699999999996, 531.0, 0.4210000000000001, 70.0])
('Europe', [249105808.0, 2038171.577, 988960.0, 5609.703000000002, 182316.0, 2080678.0, 13557.154999999997, 2626.0, 24.577999999999996, 523.0])
('Western Pacific', [85868508.0, 607137.069, 1806067.0, 5451.098000000001, 247343.0, 263345.0, 1600.8790000000001, 3555.0, 6.974, 514.0])
('Eastern Mediterranean', [22308851.0, 164737.953, 27835.0, 355.088, 2933.0, 342137.0, 1349.9960000000003, 297.0, 0.6570000000000001, 33.0])
('Africa', [9298893.0, 203666.50600000008, 10542.0, 935.7560000000001, 238.0, 174350.0, 1435.9070000000002, 42.0, 1.535, 1.0])
('Other', [764.0, 0.0, 0.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0])


In [ ]:
import numpy as np

a = np.array([1, 2])
b = np.array([2, 3])
print(a+b)

[3 5]


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext.getOrCreate('local')
sqlc = SQLContext(sc)

In [ ]:
df = sqlc.read.option("header", True) \
  .option("delimiter", ",") \
  .option("inferSchema", True) \
  .csv('/content/WHO-COVID-19-global.csv')

df.show()

+--------------------+--------------------+------------------------+----------------------------------------------+-------------------------------------+-----------------------------------------------------------+---------------------------------------+-------------------------+-----------------------------------------------+--------------------------------------+------------------------------------------------------------+----------------------------------------+
|                Name|          WHO Region|Cases - cumulative total|Cases - cumulative total per 100000 population|Cases - newly reported in last 7 days|Cases - newly reported in last 7 days per 100000 population|Cases - newly reported in last 24 hours|Deaths - cumulative total|Deaths - cumulative total per 100000 population|Deaths - newly reported in last 7 days|Deaths - newly reported in last 7 days per 100000 population|Deaths - newly reported in last 24 hours|
+--------------------+--------------------+-------------------

In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- WHO Region: string (nullable = true)
 |-- Cases - cumulative total: integer (nullable = true)
 |-- Cases - cumulative total per 100000 population: double (nullable = true)
 |-- Cases - newly reported in last 7 days: integer (nullable = true)
 |-- Cases - newly reported in last 7 days per 100000 population: double (nullable = true)
 |-- Cases - newly reported in last 24 hours: integer (nullable = true)
 |-- Deaths - cumulative total: integer (nullable = true)
 |-- Deaths - cumulative total per 100000 population: double (nullable = true)
 |-- Deaths - newly reported in last 7 days: integer (nullable = true)
 |-- Deaths - newly reported in last 7 days per 100000 population: double (nullable = true)
 |-- Deaths - newly reported in last 24 hours: integer (nullable = true)



In [ ]:
dfResutl = df.groupBy('WHO Region') \
             .sum('Cases - cumulative total') \
             .show()

+--------------------+-----------------------------+
|          WHO Region|sum(Cases - cumulative total)|
+--------------------+-----------------------------+
|              Europe|                    249105808|
|     Western Pacific|                     85868508|
|              Africa|                      9298893|
|Eastern Mediterra...|                     22308851|
|               Other|                          764|
|            Americas|                    176342137|
|     South-East Asia|                     60084208|
+--------------------+-----------------------------+



In [ ]:
cases = df.groupBy('WHO Region').sum('Cases - cumulative total','Cases - cumulative total per 100000 population','Cases - newly reported in last 7 days','Cases - newly reported in last 7 days per 100000 population','Cases - newly reported in last 24 hours')
cases.show()
cases.write.option("header",True).csv("WHO-COVID19-global/cases")


+--------------------+-----------------------------+---------------------------------------------------+------------------------------------------+----------------------------------------------------------------+--------------------------------------------+
|          WHO Region|sum(Cases - cumulative total)|sum(Cases - cumulative total per 100000 population)|sum(Cases - newly reported in last 7 days)|sum(Cases - newly reported in last 7 days per 100000 population)|sum(Cases - newly reported in last 24 hours)|
+--------------------+-----------------------------+---------------------------------------------------+------------------------------------------+----------------------------------------------------------------+--------------------------------------------+
|              Europe|                    249105808|                                        2038171.577|                                    988960|                                               5609.703000000002|              

In [ ]:
death1 = df.groupBy('WHO Region').sum('Deaths - cumulative total','Deaths - cumulative total per 100000 population','Deaths - newly reported in last 7 days','Deaths - newly reported in last 7 days per 100000 population','Deaths - newly reported in last 24 hours')
death1.show()
death1.write.option("header",True).csv("WHO-COVID19-global/deaths")


+--------------------+------------------------------+----------------------------------------------------+-------------------------------------------+-----------------------------------------------------------------+---------------------------------------------+
|          WHO Region|sum(Deaths - cumulative total)|sum(Deaths - cumulative total per 100000 population)|sum(Deaths - newly reported in last 7 days)|sum(Deaths - newly reported in last 7 days per 100000 population)|sum(Deaths - newly reported in last 24 hours)|
+--------------------+------------------------------+----------------------------------------------------+-------------------------------------------+-----------------------------------------------------------------+---------------------------------------------+
|              Europe|                       2080678|                                  13557.154999999997|                                       2626|                                               24.57799999999